In [21]:
import pandas as pd
from datetime import datetime

rawpath = "../raw/"

In [22]:
df = pd.read_excel(f"{rawpath}/2022-ES-Tabelle.xlsx", names=['signatur','jahr','datum','medienart','sprache','land','koerperschaft_1','koerperschaft_2','person', 'titel','urheber_vorlage','ort','ort_idn','umfang','bemerkung','vorh'], dtype='string')
df = df.apply(lambda x: x.str.strip())
df = df[pd.isna(df.vorh)]


/home/dbsm/.cache/pypoetry/virtualenvs/base-K0tRBB7f-py3.10/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [23]:
def pica_schreiben(row):
    pica = f"""0500 Da
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 {row.jahr}
"""
    if pd.notna(row.datum):
        if row.datum.startswith('['):
            pica += f"1110 {row.datum}*{row.jahr}$4ezth\n"
        else:
            pica += f"1110 {row.datum}$4ezth\n"



    pica += f"""1130 !040445224!
1131 !040082407!
1132 a1-analog;a2-{row.medienart};f1-text;f2-blatt
1500 /1{row.sprache}
"""
    pica += f"1700 /1{'/1'.join(row.land.split(';'))}\n"

    pica += f"3100 {row.koerperschaft_1}$BVerfasser$4aut\n"

    if pd.notna(row.koerperschaft_2):
        pica += f"3110 {row.koerperschaft_2}$BVerfasser$4aut\n"
    
    if pd.notna(row.person):
        pica += f"3010 {row.person}$BVerfasser$4aut\n"
    
    pica += f"""4000 {row.titel} / {row.urheber_vorlage}
4019 Geschäftsrundschreiben$Bobja
4046 {row.ort}
4060 {row.umfang}
4105 !1059146037!
4105 !1077661932!
"""
    if pd.notna(row.bemerkung):
        pica += f"4222 {row.bemerkung}\n"
    
    pica += f"""4700 |BSM|
5560 [RNA]Korrespondenz
5590 [Entstehungsort]
5590 {row.ort_idn}
5591 [Objektgattung]
5591 !042993512!
7001 {now.strftime('%d-%m-%y')} : x
4800 !009033645!
6710 !103243757X!
7100 {row.signatur} @ m
7109 !!DBSM/S!! ; {row.signatur}
\t\n"""
    return pica

In [2]:
feld = 'XA-CH;XA-DXDE;XD-US'
felda = feld.split(';')
"/1".join(felda)

'XA-CH/1XA-DXDE/1XD-US'

In [24]:
now = datetime.now()
for index, row in df.sample(2).iterrows():
    print(pica_schreiben(row))

0500 Da
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 1872
1110 04.10.1872$4ezth
1130 !040445224!
1131 !040082407!
1132 a1-analog;a2-hand;f1-text;f2-blatt
1500 /1ger
1700 /1XA-AAAT
3100 Deutscher Verein zur Verbreitung gemeinnütziger Kenntnisse$BVerfasser$4aut
3010 Dotzauer, Richard$BVerfasser$4aut
4000 Tritt mit dem deutschen Buchhandel in direkten Verkehr. / Rchd. Ritter v. Dotzauer; Dr. Jos. Holzamer
4019 Geschäftsrundschreiben$Bobja
4046 Prag
4060 1 Exemplar
4105 !1059146037!
4105 !1077661932!
4700 |BSM|
5560 [RNA]Korrespondenz
5590 [Entstehungsort]
5590 !040763102!
5591 [Objektgattung]
5591 !042993512!
7001 18-11-22 : x
4800 !009033645!
6710 !103243757X!
7100 Bö-GR/V/102 @ m
7109 !!DBSM/S!! ; Bö-GR/V/102
	

0500 Da
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 1847
1110 XX.09.1847$4ezth
1130 !040445224!
1131 !040082407!
1132 a1-analog;a2-druck;f1-text;f2-blatt
1500 /1ger
1700 /1XA-CH
3100 Literarische Verlags-Anst

In [25]:
now = datetime.now()
with open(f"../dat/rundschreiben-{now.strftime('%y-%m-%d-%H-%M-%S')}.dat","w") as f:
    with open(f"../dat/rundschreiben-recent.dat", 'w') as f2:
        for index, row in df.iterrows():
            f.write(pica_schreiben(row))
            f2.write(pica_schreiben(row))

with open(f"../dat/rundschreiben-recent-sample.dat", 'w') as f:
    for index, row in df.sample(5).iterrows():
        f.write(pica_schreiben(row))

In [ ]:
csfn_valnorm -u -c FCV#pica#pica#title -k VALTAB#title -s rundschreiben-recent.stat -e valtest-rundschreiben-recent.err -n -o  BVBIBIDN < rundschreiben-recent.dat.pplus 1>valtest-rundschreiben-recent.out 2>valtest-rundschreiben-recent.stderr

